# Tutorial 1: Agents
- Video: https://www.youtube.com/watch?v=twG08FeX1xs

## Functionalities:
- Task-based Agents which will break down tasks into subtasks and solve them in bite-sized portions
- Agents with registered functions as skills

# Setup Guide

## Step 1: Install AgentJo

In [1]:
# !pip install agentjo

## Step 2: Import required functions and setup relevant API keys for your LLM

In [2]:
# Set up API key and do the necessary imports
from agentjo import *
import os

# this is only if you use OpenAI as your LLM
os.environ['OPENAI_API_KEY'] = '<YOUR API KEY HERE>'

## Step 3: Define your own LLM
- Take in a `system_prompt`, `user_prompt`, and outputs llm response string

In [4]:
def llm(system_prompt: str, user_prompt: str) -> str:
    ''' Here, we use OpenAI for illustration, you can change it to your own LLM '''
    # ensure your LLM imports are all within this function
    from openai import OpenAI
    
    # define your own LLM here
    client = OpenAI()
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        temperature = 0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content

In [5]:
# Verify that llm function is working
llm(system_prompt = 'You are a classifier to classify the sentiment of a sentence', 
    user_prompt = 'It is a hot and sunny day')

'Neutral'

# 1. Agent Basics
- Create an agent by entering your agent's name and description
- Agents are task-based, so they will help generate subtasks to fulfil your main task

- Agents are made to be non-verbose, so they will just focus only on task instruction (Much more efficient compared to conversational-based agentic frameworks like AutoGen)
- Agent's interactions will be stored into `subtasks_completed` by default, which will serve as a memory buffer for future interactions

- **Inputs for Agent**:
    - **agent_name**: String. Name of agent, hinting at what the agent does
    - **agent_description**: String. Short description of what the agent does
    - **max_subtasks**: Int. Default: 5. The maximum number of subtasks the agent can have
    - **verbose**: Bool. Default: True. Whether to print out agent's intermediate thoughts
    - **llm**: Function. The LLM to be used by the Agent
<br/><br/>

- **Agent Internal Parameters**:
    - **Task**: String. The task the agent has been assigned to - Defaults to "No task assigned"
    - **Subtasks Completed**: Dict. The keys are the subtask names and the values are the result of the respective subtask
    - **Is Task Completed**: Bool. Whether the current Task is completed
<br/><br/>

- **Task Running**
    - **reset()**: Resets the Agent Internal Parameters and Subtasks Completed. You should do this at the start of every new task assigned to the Agent to minimise potential confusion of what has been done for this task versus previous tasks
    - **run(task: str, num_subtasks: int = max_subtasks)**: Performs the task. Do note that agent's state will not be reset, so if you want to reset it, call reset() prior to running this. Runs the task for **num_subtasks** steps. If not specified, we will take the **max_subtasks**.
<br/><br/>

- **Give User Output**
    - **reply_user(query: str = '', stateful: bool = True)**: Using all information from subtasks, give a reply about the `query` to the user. If `query` is not given, then it replies based on the current task the agent is doing. If `stateful` is True, saves this query and reply into `subtasks_completed`
<br/><br/>

     - **answer(query, output_format = {'Answer': 'Concise Answer'})**: Using all information from subtasks, give a reply about the `query` to the user. Answers in the given `output_format`
<br/><br/>

    
- **Check status of Agent**:
    - **status()**: Lists out Agent Name, Agent Description, Available Functions (default function is to use the LLM), Task, Subtasks Completed and Is Task Completed
    
## Example Agent Creation
```python
my_agent = Agent('Helpful assistant', 'You are a generalist agent')
```

## Example Agent Task Running - Split the assigned task into subtasks and execute each of them

```python
output = my_agent.run('Give me 5 words rhyming with cool, and make a 4-sentence poem using them')
```

`Subtask identified: Find 5 words that rhyme with 'cool'`

`Getting LLM to perform the following task: Find 5 words that rhyme with 'cool'`
> pool, rule, fool, tool, school

`Subtask identified: Compose a 4-sentence poem using the words 'pool', 'rule', 'fool', 'tool', and 'school'`

`Getting LLM to perform the following task: Compose a 4-sentence poem using the words 'pool', 'rule', 'fool', 'tool', and 'school'`
> In the school, the golden rule is to never be a fool. Use your mind as a tool, and always follow the pool.

`Task completed successfully!`

## Check Agent's Status
```python
my_agent.status()
```

`Agent Name: Helpful assistant`

`Agent Description: You are a generalist agent`

`Available Functions: ['use_llm', 'end_task']`

`Task: Give me 5 words rhyming with cool, and make a 4-sentence poem using them`

`Subtasks Completed:`

`Subtask: Find 5 words that rhyme with 'cool'`

`pool, rule, fool, tool, school`

`Subtask: Compose a 4-sentence poem using the words 'pool', 'rule', 'fool', 'tool', and 'school'`

`In the school, the golden rule is to never be a fool. Use your mind as a tool, and always follow the pool.`

`Is Task Completed: True`

## Example Agent Reply to User - Reference the subtasks' output to answer the user's query
```python
output = my_agent.reply_user()
```

`
Here are 5 words that rhyme with "cool": pool, rule, fool, tool, school. Here is a 4-sentence poem using these words: "In the school, the golden rule is to never be a fool. Use your mind as a tool, and always follow the pool."
`

## Create Agent

In [6]:
# Create your agent by specifying name and description
my_agent = Agent('Helpful assistant', 'You are a generalist agent', llm = llm)

In [7]:
# Show the agent status - By default agent comes equipped with default function `use_llm` which queries the llm
# end_task is to end the current task if it is completed
my_agent.status()

Agent Name: Helpful assistant
Agent Description: You are a generalist agent
Available Functions: ['use_llm', 'end_task']
Shared Variables: ['agent']
Task: No task assigned
Subtasks Completed: None
Is Task Completed: False


## Automatic Running of Task

In [8]:
# Do the task by subtasks. This does generation to fulfil task
my_agent.reset()
output = my_agent.run('Give me 5 words rhyming with cool, and then make a 4-sentence poem using them')

Observation: No subtasks have been completed yet for the assigned task of generating rhyming words and a poem.
Thoughts: To complete the assigned task, I need to first generate five words that rhyme with "cool". Once I have those words, I can then create a four-sentence poem using them.
Subtask identified: Generate five words that rhyme with "cool".
Getting LLM to perform the following task: Generate five words that rhyme with "cool".
> The five words that rhyme with "cool" are: 1. School, 2. Pool, 3. Tool, 4. Fool, 5. Rule. These words share the same ending sound as "cool," making them perfect rhymes. Each word can be used in various contexts, such as "school" referring to an educational institution, "pool" as a recreational water area, "tool" indicating an instrument for work, "fool" describing a person who acts unwisely, and "rule" denoting a guideline or principle.

Observation: Five words that rhyme with "cool" have been successfully generated: school, pool, tool, fool, and rule. 

In [9]:
# visualise the subtask outputs of the task
print(output)

['The five words that rhyme with "cool" are: 1. School, 2. Pool, 3. Tool, 4. Fool, 5. Rule. These words share the same ending sound as "cool," making them perfect rhymes. Each word can be used in various contexts, such as "school" referring to an educational institution, "pool" as a recreational water area, "tool" indicating an instrument for work, "fool" describing a person who acts unwisely, and "rule" denoting a guideline or principle.', 'In a bright and lively school, where laughter is the rule,  \nChildren gather by the pool, splashing like a playful fool.  \nWith every game, they find a tool, to build their dreams and break the mold,  \nCreating memories, precious and cool, as stories of their youth unfold.']


In [10]:
# see the updated agent status
my_agent.status()

Agent Name: Helpful assistant
Agent Description: You are a generalist agent
Available Functions: ['use_llm', 'end_task']
Shared Variables: ['agent']
Task: Give me 5 words rhyming with cool, and then make a 4-sentence poem using them
Subtasks Completed:
Subtask: Generate five words that rhyme with "cool".
The five words that rhyme with "cool" are: 1. School, 2. Pool, 3. Tool, 4. Fool, 5. Rule. These words share the same ending sound as "cool," making them perfect rhymes. Each word can be used in various contexts, such as "school" referring to an educational institution, "pool" as a recreational water area, "tool" indicating an instrument for work, "fool" describing a person who acts unwisely, and "rule" denoting a guideline or principle.

Subtask: Compose a four-sentence poem that incorporates the words school, pool, tool, fool, and rule, ensuring that each word is used meaningfully within the context of the poem.
In a bright and lively school, where laughter is the rule,  
Children gat

In [11]:
# Generates a meaningful reply to the user about the task according to its current state. Functions like a QA bot
# The reply will go into subtasks_completed to store the conversation with the user for future context
output = my_agent.reply_user()

The five words that rhyme with "cool" are: 1. School, 2. Pool, 3. Tool, 4. Fool, 5. Rule. These words share the same ending sound as "cool," making them perfect rhymes. Each word can be used in various contexts, such as "school" referring to an educational institution, "pool" as a recreational water area, "tool" indicating an instrument for work, "fool" describing a person who acts unwisely, and "rule" denoting a guideline or principle. 

Here is a four-sentence poem incorporating these words: 
In a bright and lively school, where laughter is the rule,  
Children gather by the pool, splashing like a playful fool.  
With every game, they find a tool, to build their dreams and break the mold,  
Creating memories, precious and cool, as stories of their youth unfold.


In [12]:
# You can also ask your questions to the agent, and the agent will reply according to its current state. Functions like a QA bot
# Here, we set stateful is false because we do not want this to go into subtasks_completed
output = my_agent.reply_user('Where is the pool?', stateful = False)

The pool is mentioned in the context of a lively school where children gather to splash and play. It serves as a recreational area for the children, providing a space for fun and enjoyment. The poem describes the pool as a place where laughter is abundant, and children engage in playful activities, highlighting its importance in their school experience.


In [13]:
# see the updated agent status
my_agent.status()

Agent Name: Helpful assistant
Agent Description: You are a generalist agent
Available Functions: ['use_llm', 'end_task']
Shared Variables: ['agent']
Task: Give me 5 words rhyming with cool, and then make a 4-sentence poem using them
Subtasks Completed:
Subtask: Generate five words that rhyme with "cool".
The five words that rhyme with "cool" are: 1. School, 2. Pool, 3. Tool, 4. Fool, 5. Rule. These words share the same ending sound as "cool," making them perfect rhymes. Each word can be used in various contexts, such as "school" referring to an educational institution, "pool" as a recreational water area, "tool" indicating an instrument for work, "fool" describing a person who acts unwisely, and "rule" denoting a guideline or principle.

Subtask: Compose a four-sentence poem that incorporates the words school, pool, tool, fool, and rule, ensuring that each word is used meaningfully within the context of the poem.
In a bright and lively school, where laughter is the rule,  
Children gat

In [14]:
my_agent.print_functions()

Name: use_llm
Description: For general tasks. Used only when no other function can do the task
Input: []
Output: {'Output': 'Output of LLM'}

Name: end_task
Description: Passes the final output to the user
Input: []
Output: {}



In [15]:
my_agent.reset()

In [16]:
my_agent.status()

Agent Name: Helpful assistant
Agent Description: You are a generalist agent
Available Functions: ['use_llm', 'end_task']
Shared Variables: ['agent']
Task: No task assigned
Subtasks Completed: None
Is Task Completed: False


In [17]:
# when you change the task, we will set the task completed to False so you can do it again. 
# Do note that the earlier subtasks_history might affect your new task and should be reset if it is not relevant
my_agent.reset()
output = my_agent.run('Give me 5 words rhyming with task, and then make a 4-sentence poem using them')

Observation: No subtasks have been completed yet for the assigned task of generating rhyming words and a poem.
Thoughts: To complete the assigned task, I need to first find five words that rhyme with "task". Once I have those words, I can then create a four-sentence poem using them.
Subtask identified: Identify five words that rhyme with "task" to fulfill the first part of the assigned task.
Getting LLM to perform the following task: Identify five words that rhyme with "task" to fulfill the first part of the assigned task.
> To fulfill the assigned subtask of identifying five words that rhyme with "task," I analyzed the phonetic structure of the word. The ending sound of "task" is crucial for finding suitable rhymes. After considering various options, I identified the following five words that rhyme with "task": "mask," "flask," "brask," "ask," and "cask." Each of these words shares the same ending sound, making them perfect rhymes for "task." This outcome not only meets the requiremen

In [18]:
# see the updated agent staus
my_agent.status()

Agent Name: Helpful assistant
Agent Description: You are a generalist agent
Available Functions: ['use_llm', 'end_task']
Shared Variables: ['agent']
Task: Give me 5 words rhyming with task, and then make a 4-sentence poem using them
Subtasks Completed:
Subtask: Identify five words that rhyme with "task" to fulfill the first part of the assigned task.
To fulfill the assigned subtask of identifying five words that rhyme with "task," I analyzed the phonetic structure of the word. The ending sound of "task" is crucial for finding suitable rhymes. After considering various options, I identified the following five words that rhyme with "task": "mask," "flask," "brask," "ask," and "cask." Each of these words shares the same ending sound, making them perfect rhymes for "task." This outcome not only meets the requirement of the subtask but also provides a variety of options that can be useful in different contexts, such as poetry, songwriting, or word games.

Subtask: Compose a 4-sentence poem 

In [19]:
# possible to ask a query and get an answer in structured format
my_agent.answer("What are the five words", 
                output_format = {"Words": "type: list", 
                 "Words and their meaning": "type: dict"})

{'Words': ['mask', 'flask', 'brask', 'ask', 'cask'],
 'Words and their meaning': {'mask': 'a covering for all or part of the face',
  'flask': 'a container for liquids, typically with a narrow neck',
  'brask': 'a term used in some dialects, often referring to a lively or boisterous person',
  'ask': 'to request information or an answer',
  'cask': 'a barrel-shaped container, typically used for storing liquids'}}

## Running of Task Step by Step

In [20]:
# Create your agent by specifying name and description
my_agent = Agent('Number Expert', 'You are great with numbers', llm = llm)

In [21]:
# Runs a task for 1 step by editing num_subtasks variable to 1
output = my_agent.run('List me three random numbers from 1 to 50, then give me their sum', num_subtasks = 1)

Observation: No subtasks have been completed yet for the assigned task of listing three random numbers and calculating their sum.
Thoughts: To complete the assigned task, I need to generate three random numbers between 1 and 50 and then calculate their sum.
Subtask identified: Generate three random numbers from 1 to 50 and calculate their sum.
Getting LLM to perform the following task: Generate three random numbers from 1 to 50 and calculate their sum.
> I generated three random numbers from the range of 1 to 50. The numbers selected are 12, 27, and 35. To find their sum, I added these numbers together: 12 + 27 + 35 = 74. Therefore, the sum of the three random numbers is 74.



In [22]:
# visualise the outputs of the task at the subtask level
print(output)

['I generated three random numbers from the range of 1 to 50. The numbers selected are 12, 27, and 35. To find their sum, I added these numbers together: 12 + 27 + 35 = 74. Therefore, the sum of the three random numbers is 74.']


In [23]:
# Visualise the first subtask
my_agent.status()

Agent Name: Number Expert
Agent Description: You are great with numbers
Available Functions: ['use_llm', 'end_task']
Shared Variables: ['agent']
Task: List me three random numbers from 1 to 50, then give me their sum
Subtasks Completed:
Subtask: Generate three random numbers from 1 to 50 and calculate their sum.
I generated three random numbers from the range of 1 to 50. The numbers selected are 12, 27, and 35. To find their sum, I added these numbers together: 12 + 27 + 35 = 74. Therefore, the sum of the three random numbers is 74.

Is Task Completed: False


In [24]:
# Runs the task for another step
output = my_agent.run('List me three random numbers from 1 to 50, then give me their sum', num_subtasks = 1)

Observation: I have generated three random numbers: 12, 27, and 35, and calculated their sum to be 74.
Thoughts: The Assigned Task is now complete as I have provided both the three random numbers and their sum.
Subtask identified: End Task
Task completed successfully!



In [25]:
# visualise the outputs of the task at the subtask level
print(output)

['I generated three random numbers from the range of 1 to 50. The numbers selected are 12, 27, and 35. To find their sum, I added these numbers together: 12 + 27 + 35 = 74. Therefore, the sum of the three random numbers is 74.']


In [26]:
# Visualise the first and second subtasks
my_agent.status()

Agent Name: Number Expert
Agent Description: You are great with numbers
Available Functions: ['use_llm', 'end_task']
Shared Variables: ['agent']
Task: List me three random numbers from 1 to 50, then give me their sum
Subtasks Completed:
Subtask: Generate three random numbers from 1 to 50 and calculate their sum.
I generated three random numbers from the range of 1 to 50. The numbers selected are 12, 27, and 35. To find their sum, I added these numbers together: 12 + 27 + 35 = 74. Therefore, the sum of the three random numbers is 74.

Is Task Completed: True


In [27]:
# if task already completed, then no need to do further
output = my_agent.run('List me three random numbers from 1 to 50, then give me their sum', num_subtasks = 1)

Observation: I generated three random numbers from the range of 1 to 50: 12, 27, and 35, and calculated their sum to be 74.
Thoughts: The Assigned Task is now complete as I have provided both the three random numbers and their sum.
Subtask identified: End Task
Task completed successfully!



In [28]:
# Generates a meaningful reply to the user
output = my_agent.reply_user()

I generated three random numbers from the range of 1 to 50. The numbers selected are 12, 27, and 35. To find their sum, I added these numbers together: 12 + 27 + 35 = 74. Therefore, the three random numbers are 12, 27, and 35, and their sum is 74.


# 2. Power Up your Agents - Bring in Functions (aka Tools)
- First define the functions, either using class `Function` (see Tutorial 0), or just any Python function with input and output types defined in the signature and with a docstring
- After creating your agent, use `assign_functions` to assign a list of functions of class `Function`, or general Python functions (which will be converted to AsyncFunction)
- Function names will be automatically inferred if not specified
- Proceed to run tasks by using `run()`

```python
# This is an example of an LLM-based function (see Tutorial 0)
sentence_style = Function(fn_description = 'Output a sentence with words <var1> and <var2> in the style of <var3>', 
                         output_format = {'output': 'sentence'},
                         fn_name = 'sentence_with_objects_entities_emotion',
                         llm = llm)

# This is an example of an external user-defined function (see Tutorial 0)
def binary_to_decimal(binary_number: str) -> int:
    '''Converts binary_number to integer of base 10'''
    return int(str(binary_number), 2)

# Initialise your Agent
my_agent = Agent('Helpful assistant', 'You are a generalist agent')

# Assign the functions
my_agent.assign_functions([sentence_style, binary_to_decimal])

# Run the Agent
output = my_agent.run('First convert binary string 1001 to a number, then generate me a happy sentence with that number and a ball')
```

`Subtask identified: Convert the binary number 1001 to decimal`
`Calling function binary_to_decimal with parameters {'x': '1001'}`

> {'output1': 9}

`Subtask identified: Generate a happy sentence with the decimal number and a ball`
`Calling function sentence_with_objects_entities_emotion with parameters {'obj': '9', 'entity': 'ball', 'emotion': 'happy'}`

> {'output': 'I am so happy with my 9 balls.'}

`Task completed successfully!`

- Approach 1: Automatically Run your agent using `run()`

- Approach 2: Manually select and use functions for your task
    - **select_function(task: str)**: Based on the task, output the next function name and input parameters
    - **use_function(function_name: str, function_params: dict, subtask: str = '', stateful: bool = True)**: Uses the function named `function_name` with `function_params`. `stateful` controls whether the output of this function will be saved to `subtasks_completed` under the key of `subtask`
<br/><br/>

- **Assign/Remove Functions**:
    - **assign_functions(function_list: list)**: Assigns a list of functions to the agent
    - **remove_function(function_name: str)**: Removes function named function_name from the list of assigned functions
<br/><br/>

- **Show Functions**:
    - **list_functions()**: Returns the list of functions of the agent
    - **print_functions()**: Prints the list of functions of the agent
<br/><br/>

In [29]:
# Example Internal Function
sentence_style = Function(fn_description = 'Output a sentence with <obj> and <entity> in the style of <emotion>', 
                     output_format = {'output': 'sentence'}, fn_name = 'sentence_with_objects_entities_emotion',
                         llm = llm)

In [30]:
print(sentence_style)

Description: Output a sentence with <obj> and <entity> in the style of <emotion>
Input: ['obj', 'entity', 'emotion']
Output: {'output': 'sentence'}



In [31]:
# Example External Function
def binary_to_decimal(binary_string: str) -> int:
    '''Converts binary_string to integer of base 10'''
    return int(str(binary_string), 2)

In [32]:
# Initialise your agent
my_agent = Agent('Helpful assistant', 'You are a generalist agent', llm = llm)

In [33]:
# Assign functions
my_agent.assign_functions(function_list = [sentence_style, binary_to_decimal])

In [34]:
# Show the functions the agent has
my_agent.print_functions()

Name: use_llm
Description: For general tasks. Used only when no other function can do the task
Input: []
Output: {'Output': 'Output of LLM'}

Name: end_task
Description: Passes the final output to the user
Input: []
Output: {}

Name: sentence_with_objects_entities_emotion
Description: Output a sentence with <obj> and <entity> in the style of <emotion>
Input: ['obj', 'entity', 'emotion']
Output: {'output': 'sentence'}

Name: binary_to_decimal
Description: Converts <binary_string: str> to integer of base 10
Input: ['binary_string']
Output: {'output_1': 'int'}



### Approach 1: Automatic Running of Task

In [35]:
my_agent.reset()
output = my_agent.run('First convert binary string 1001 to a number, then generate me a happy sentence with that number and a ball')

Observation: No subtasks have been completed yet for the assigned task, which involves converting a binary string to a number and generating a happy sentence.
Thoughts: To complete the assigned task, I need to first convert the binary string 1001 into its decimal equivalent. Once I have that number, I can then use it to create a happy sentence that includes the number and a ball.
Subtask identified: Convert the binary string 1001 to its decimal equivalent.
Calling function binary_to_decimal with parameters {'binary_string': '1001'}
> {'output_1': 9}

Observation: The binary string 1001 has been successfully converted to the decimal number 9.
Thoughts: Now that I have the number 9, I need to create a happy sentence that includes the number 9 and a ball.
Subtask identified: Generate a happy sentence that incorporates the number 9 and a ball.
Calling function sentence_with_objects_entities_emotion with parameters {'obj': 9, 'entity': 'ball', 'emotion': 'happy'}
> {'output': 'The ball boun

In [36]:
my_agent.status()

Agent Name: Helpful assistant
Agent Description: You are a generalist agent
Available Functions: ['use_llm', 'end_task', 'sentence_with_objects_entities_emotion', 'binary_to_decimal']
Shared Variables: ['agent']
Task: First convert binary string 1001 to a number, then generate me a happy sentence with that number and a ball
Subtasks Completed:
Subtask: binary_to_decimal(binary_string="1001")
{'output_1': 9}

Subtask: sentence_with_objects_entities_emotion(obj=9, entity="ball", emotion="happy")
{'output': 'The ball bounced joyfully as the children played with it, filling the air with laughter and happiness.'}

Is Task Completed: True


In [37]:
# give a response to user
output = my_agent.reply_user()

The binary string "1001" has been converted to the decimal number 9. Using this number, I generated a happy sentence: "The ball bounced joyfully as the children played with it, filling the air with laughter and happiness." This sentence incorporates the number 9, the entity "ball," and the emotion "happy."


In [38]:
# query according to what you need
output = my_agent.reply_user('Output only the sentence')

The ball bounced joyfully as the children played with it, filling the air with laughter and happiness.


In [39]:
my_agent.status()

Agent Name: Helpful assistant
Agent Description: You are a generalist agent
Available Functions: ['use_llm', 'end_task', 'sentence_with_objects_entities_emotion', 'binary_to_decimal']
Shared Variables: ['agent']
Task: First convert binary string 1001 to a number, then generate me a happy sentence with that number and a ball
Subtasks Completed:
Subtask: binary_to_decimal(binary_string="1001")
{'output_1': 9}

Subtask: sentence_with_objects_entities_emotion(obj=9, entity="ball", emotion="happy")
{'output': 'The ball bounced joyfully as the children played with it, filling the air with laughter and happiness.'}

Subtask: First convert binary string 1001 to a number, then generate me a happy sentence with that number and a ball
The binary string "1001" has been converted to the decimal number 9. Using this number, I generated a happy sentence: "The ball bounced joyfully as the children played with it, filling the air with laughter and happiness." This sentence incorporates the number 9, th

In [40]:
my_agent.subtasks_completed

{'binary_to_decimal(binary_string="1001")': {'output_1': 9},
 'sentence_with_objects_entities_emotion(obj=9, entity="ball", emotion="happy")': {'output': 'The ball bounced joyfully as the children played with it, filling the air with laughter and happiness.'},
 'First convert binary string 1001 to a number, then generate me a happy sentence with that number and a ball': 'The binary string "1001" has been converted to the decimal number 9. Using this number, I generated a happy sentence: "The ball bounced joyfully as the children played with it, filling the air with laughter and happiness." This sentence incorporates the number 9, the entity "ball," and the emotion "happy."',
 'Output only the sentence': 'The ball bounced joyfully as the children played with it, filling the air with laughter and happiness.'}

In [41]:
my_agent.thoughts

[{'Observation': 'No subtasks have been completed yet for the assigned task, which involves converting a binary string to a number and generating a happy sentence.',
  'Thoughts': 'To complete the assigned task, I need to first convert the binary string 1001 into its decimal equivalent. Once I have that number, I can then use it to create a happy sentence that includes the number and a ball.',
  'Current Subtask': 'Convert the binary string 1001 to its decimal equivalent.',
  'Equipped Function Name': 'binary_to_decimal',
  'Equipped Function Inputs': {'binary_string': '1001'}},
 {'Observation': 'The binary string 1001 has been successfully converted to the decimal number 9.',
  'Thoughts': 'Now that I have the number 9, I need to create a happy sentence that includes the number 9 and a ball.',
  'Current Subtask': 'Generate a happy sentence that incorporates the number 9 and a ball.',
  'Equipped Function Name': 'sentence_with_objects_entities_emotion',
  'Equipped Function Inputs': {

### Approach 2: Manual Selection and Running of Functions
- If you want to just see what the agent would choose for a hypothetical task, use `select_function`. This will not update the internal state, and will output function name and function params
- If you want specificity in the process, you can just execute the agent's functions yourself using `use_function`
- `use_function`: Uses an agent's function using name and params. Note that by default there will be updating of subtasks_completed when performing the function. In order not to update subtasks_completed, set `stateful = False`

In [42]:
# this should call generate_sentence_with_emotion
my_agent.reset()
function_name, function_params = my_agent.select_function(
    task = 'Output a sentence with bell, dog and happy')
print(f'Selected function name: {function_name}\nSelected function params: {function_params}')

my_agent.use_function(function_name, function_params, stateful = False)

Observation: The Assigned Task requires generating a sentence that includes the words "bell," "dog," and conveys a sense of "happy."
Thoughts: To complete the Assigned Task, I need to use the equipped function that can create a sentence incorporating the specified objects and emotion.
Selected function name: sentence_with_objects_entities_emotion
Selected function params: {'obj': 'bell', 'entity': 'dog', 'emotion': 'happy'}
Calling function sentence_with_objects_entities_emotion with parameters {'obj': 'bell', 'entity': 'dog', 'emotion': 'happy'}
> {'output': 'The dog joyfully barked at the ringing bell, its tail wagging with excitement.'}



{'output': 'The dog joyfully barked at the ringing bell, its tail wagging with excitement.'}

In [43]:
# this should call binary_to_decimal
my_agent.reset()
function_name, function_params = my_agent.select_function(
    task = 'What is the decimal representation of binary number 101?')
print(f'Selected function name: {function_name}\nSelected function params: {function_params}')

my_agent.use_function(function_name, function_params, stateful = False)

Observation: The binary number 101 has been identified as the input for conversion to its decimal representation.
Thoughts: To complete the task, I need to convert the binary string 101 into its decimal equivalent using the appropriate function.
Selected function name: binary_to_decimal
Selected function params: {'binary_string': '101'}
Calling function binary_to_decimal with parameters {'binary_string': '101'}
> {'output_1': 5}



{'output_1': 5}

In [44]:
# this should call use_llm
my_agent.reset()
function_name, function_params = my_agent.select_function(
    task = 'Research on the benefits of exercise based on LLM')
print(f'Selected function name: {function_name}\nSelected function params: {function_params}')

my_agent.use_function(function_name, function_params, stateful = False)

Observation: I have researched various aspects of exercise and its benefits using the LLM, gathering information on physical, mental, and emotional health improvements.
Thoughts: To complete the remainder of the task, I need to synthesize the gathered information into a coherent summary that highlights the key benefits of exercise.
Selected function name: use_llm
Selected function params: {'instruction': 'Summarize the benefits of exercise based on the research findings from the LLM.'}
Getting LLM to perform the following task: Summarize the benefits of exercise based on the research findings from the LLM.
> Exercise has numerous benefits supported by extensive research findings. Firstly, regular physical activity improves cardiovascular health by strengthening the heart and enhancing circulation, which can reduce the risk of heart disease. Secondly, exercise is known to aid in weight management by burning calories and increasing metabolism, which is crucial for preventing obesity. Add

{'Detailed Outcome': 'Exercise has numerous benefits supported by extensive research findings. Firstly, regular physical activity improves cardiovascular health by strengthening the heart and enhancing circulation, which can reduce the risk of heart disease. Secondly, exercise is known to aid in weight management by burning calories and increasing metabolism, which is crucial for preventing obesity. Additionally, engaging in physical activity can enhance mental health by reducing symptoms of anxiety and depression, as it promotes the release of endorphins, often referred to as "feel-good" hormones. Furthermore, exercise has been linked to improved cognitive function, including better memory and concentration, which can be particularly beneficial as individuals age. It also strengthens muscles and bones, reducing the risk of osteoporosis and frailty in older adults. Lastly, regular exercise can improve sleep quality, leading to better overall health and well-being. Overall, the research

# 3. AsyncAgent

- `AsyncAgent` works the same way as `Agent`, only much faster due to parallelisation of tasks
- It can only be assigned functions of class `AsyncFunction`, or general Python functions (which will be converted to AsyncFunction)
- If you define your own `AsyncFunction`, you should define the fn_name as well if it is not an External Function
- As a rule of thumb, just add the `await` keyword to any function that you run with the `AsyncAgent`

#### Example LLM in Async Mode
```python
async def llm_async(system_prompt: str, user_prompt: str):
    ''' Here, we use OpenAI for illustration, you can change it to your own LLM '''
    # ensure your LLM imports are all within this function
    from openai import AsyncOpenAI
    
    # define your own LLM here
    client = AsyncOpenAI()
    response = await client.chat.completions.create(
        model='gpt-3.5-turbo',
        temperature = 0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content
```

#### Example Agentic Workflow
```python
# This is an example of an LLM-based function (see Tutorial 0)
sentence_style = AsyncFunction(fn_description = 'Output a sentence with words <var1> and <var2> in the style of <var3>', 
                         output_format = {'output': 'sentence'},
                         fn_name = 'sentence_with_objects_entities_emotion', # you must define fn_name for LLM-based functions
                         llm = llm_async) # use an async LLM function

# This is an example of an external user-defined function (see Tutorial 0)
def binary_to_decimal(binary_number: str) -> int:
    '''Converts binary_number to integer of base 10'''
    return int(str(binary_number), 2)

# Initialise your Agent
my_agent = AsyncAgent('Helpful assistant', 'You are a generalist agent', llm = llm_async)

# Assign the functions
my_agent.assign_functions([sentence_style, binary_to_decimal])

# Run the Agent
output = await my_agent.run('Generate me a happy sentence with a number and a ball. The number is b1001 converted to decimal')
```

In [45]:
# Define an Async LLM function
async def llm_async(system_prompt: str, user_prompt: str):
    ''' Here, we use OpenAI for illustration, you can change it to your own LLM '''
    # ensure your LLM imports are all within this function
    from openai import AsyncOpenAI
    
    # define your own LLM here
    client = AsyncOpenAI()
    response = await client.chat.completions.create(
        model='gpt-4o-mini',
        temperature = 0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content

In [46]:
# This is an example of an LLM-based function (see Tutorial 0)
sentence_style = AsyncFunction(fn_description = 'Output a sentence with words <var1> and <var2> in the style of <var3>', 
                         output_format = {'output': 'sentence'},
                         fn_name = 'sentence_with_objects_entities_emotion', # you must define fn_name for LLM-based functions
                         llm = llm_async) # you must use an Async LLM function

# This is an example of an external user-defined function (see Tutorial 0)
def binary_to_decimal(binary_number: str) -> int:
    '''Converts binary_number to integer of base 10'''
    return int(str(binary_number), 2)

# Initialise your Agent
my_agent = AsyncAgent('Helpful assistant', 'You are a generalist agent', llm = llm_async)

# Assign the functions
my_agent.assign_functions([sentence_style, binary_to_decimal])

# Run the Agent
output = await my_agent.run('First convert binary string 1001 to a number, then generate me a happy sentence with that number and a ball')

Observation: No subtasks have been completed yet for the assigned task, which involves converting a binary string to a number and generating a sentence.
Thoughts: To complete the assigned task, I need to first convert the binary string 1001 into its decimal equivalent. Once I have that number, I can then use it to create a happy sentence that includes the number and a ball.
Subtask identified: Convert the binary string 1001 to its decimal equivalent.
Calling function binary_to_decimal with parameters {'binary_number': '1001'}
> {'output_1': 9}

Observation: The binary string 1001 has been successfully converted to the number 9.
Thoughts: Now that I have the number 9, I need to generate a happy sentence that includes the number 9 and a ball.
Subtask identified: Use the sentence_with_objects_entities_emotion function to create a happy sentence incorporating the number 9 and a ball.
Calling function sentence_with_objects_entities_emotion with parameters {'var1': 9, 'var2': 'ball', 'var3':

In [47]:
output = await my_agent.reply_user()

The binary string "1001" has been successfully converted to its decimal equivalent, which is 9. Following this conversion, a happy sentence was generated that incorporates the number 9 and the object "ball". The resulting sentence is: "I am so happy to see the 9 colorful balls bouncing around!" This sentence effectively combines the number derived from the binary conversion with the specified object and emotion.


In [49]:
await my_agent.answer("What is the number?",
                      output_format = {"Binary": "type: str", 
                                       "Integer": "type: int"})

{'Binary': '1001', 'Integer': 9}

In [50]:
my_agent.status()

Agent Name: Helpful assistant
Agent Description: You are a generalist agent
Available Functions: ['use_llm', 'end_task', 'sentence_with_objects_entities_emotion', 'binary_to_decimal']
Shared Variables: ['agent']
Task: First convert binary string 1001 to a number, then generate me a happy sentence with that number and a ball
Subtasks Completed:
Subtask: binary_to_decimal(binary_number="1001")
{'output_1': 9}

Subtask: sentence_with_objects_entities_emotion(var1=9, var2="ball", var3="happy")
{'output': 'I am so happy to see the 9 colorful balls bouncing around!'}

Subtask: First convert binary string 1001 to a number, then generate me a happy sentence with that number and a ball
The binary string "1001" has been successfully converted to its decimal equivalent, which is 9. Following this conversion, a happy sentence was generated that incorporates the number 9 and the object "ball". The resulting sentence is: "I am so happy to see the 9 colorful balls bouncing around!" This sentence effe

In [51]:
# this should call generate_sentence_with_emotion
my_agent.reset()
function_name, function_params = await my_agent.select_function(
    task = 'Output a sentence with bell, dog and happy')
print(f'Selected function name: {function_name}\nSelected function params: {function_params}')

await my_agent.use_function(function_name, function_params, stateful = False)

Observation: The task requires generating a sentence that includes the words "bell," "dog," and "happy."
Thoughts: To complete the task, I need to use the equipped function that can create a sentence incorporating the specified words in a stylistic manner.
Selected function name: sentence_with_objects_entities_emotion
Selected function params: {'var1': 'bell', 'var2': 'dog', 'var3': 'joyful'}
Calling function sentence_with_objects_entities_emotion with parameters {'var1': 'bell', 'var2': 'dog', 'var3': 'joyful'}
> {'output': 'The joyful bell rang as the dog wagged its tail in delight.'}



{'output': 'The joyful bell rang as the dog wagged its tail in delight.'}

# Saving and Loading Agents
Sometimes you want to configure your agents and save them and load them elsewhere, while maintaining the current agent state

- When you use the `save_agent` function, we store the entire agent's internal state, include name, description, list of functions, subtasks history and all other internal variables into a pickle file
- When you use the `load_agent` function, and we will load the entire agent saved in the pickle file into the existing agent

Key functions:
- **save_agent(pickle_file_name: str)**: Saves the agent's internal parameters to a pickle file named pickle_file_name (include .pkl), returns the pickle file
- **load_agent(pickle_file_name: str)**: Loads the agent's internal parameters from a pickle file named pickle_file_name (include .pkl), returns loaded agent

#### Example 1: Saving Agent
```python
my_agent.save_agent('myagent.pkl')
```

#### Example Output
```Agent saved to myagent.pkl```

#### Example 2: Loading Agent
```python
new_agent = Agent().load_agent('myagent.pkl')
```

#### Example Output
```Agent loaded from myagent.pkl```

In [52]:
# run one task to add something to subtasks_completed
my_agent.reset()
output = my_agent.run('What is the decimal representation of binary number 101?')

In [53]:
# see status of agent before saving
my_agent.status()

Agent Name: Helpful assistant
Agent Description: You are a generalist agent
Available Functions: ['use_llm', 'end_task', 'sentence_with_objects_entities_emotion', 'binary_to_decimal']
Shared Variables: ['agent']
Task: No task assigned
Subtasks Completed: None
Is Task Completed: False


In [54]:
# save agent
my_agent.save_agent('myagent.pkl')

Agent saved to myagent.pkl


In [55]:
# load agent using load_agent code
new_agent = Agent().load_agent('myagent.pkl')

Agent loaded from myagent.pkl


In [56]:
# see status of loaded agent (which also includes what is saved in subtasks_completed, and shared_variables)
new_agent.status()

Agent Name: Helpful assistant
Agent Description: You are a generalist agent
Available Functions: ['use_llm', 'end_task', 'sentence_with_objects_entities_emotion', 'binary_to_decimal']
Shared Variables: ['agent']
Task: No task assigned
Subtasks Completed: None
Is Task Completed: False
